In [1]:
!head -105 '/afs/crc.nd.edu/group/dmsquare/vol1/data/WIKIPEDIA/wikipedia_dump/enwiki-20220101-pages-articles-multistream-index.txt' | tail -10

600:589:Ashmore And Cartier Islands
600:590:Austin (disambiguation)
600:593:Animation
600:594:Apollo
600:595:Andre Agassi
676575:596:Artificial languages
676575:597:Austroasiatic languages
676575:598:Afro-asiatic languages
676575:599:Afroasiatic languages
676575:600:Andorra


In [2]:
index_filename = '/afs/crc.nd.edu/group/dmsquare/vol1/data/WIKIPEDIA/wikipedia_dump/enwiki-20220101-pages-articles-multistream-index.txt'
wiki_filename = '/afs/crc.nd.edu/group/dmsquare/vol1/data/WIKIPEDIA/wikipedia_dump/enwiki-20220101-pages-articles-multistream.xml.bz2'


In [3]:
import csv

def search_index(search_term, index_filename):
    byte_flag = False
    data_length = start_byte = 0
    index_file = open(index_filename, 'r')
    csv_reader = csv.reader(index_file, delimiter=':')
    for line in csv_reader:
        if not byte_flag and search_term == line[2]:
            start_byte = int(line[0])
            byte_flag = True
        elif byte_flag and int(line[0]) != start_byte:
            data_length = int(line[0]) - start_byte
            break
    index_file.close()
    return start_byte, data_length

In [5]:
search_term = 'Hajee Mohammad Danesh Science & Technology University'
index_filename = '/afs/crc.nd.edu/group/dmsquare/vol1/data/WIKIPEDIA/wikipedia_dump/enwiki-20220101-pages-articles-multistream-index.txt'
search_index(search_term, index_filename)

(0, 0)

In [7]:
import bz2
import shutil

def decompress_chunk(wiki_filename, start_byte, data_length):
    temp_filename = 'chunk.bz2'
    decomp_filename = 'chunk.xml'

    with open(wiki_filename, 'rb') as wiki_file:
        wiki_file.seek(start_byte)
        data = wiki_file.read(data_length)

    with open(temp_filename, 'wb') as temp_file:
        temp_file.write(data)

    with bz2.BZ2File(temp_filename) as fr, open(decomp_filename,"wb") as fw:
        shutil.copyfileobj(fr,fw)

    return decomp_filename

In [11]:
search_term = 'Donald Trump'
index_filename = '/afs/crc.nd.edu/group/dmsquare/vol1/data/WIKIPEDIA/wikipedia_dump/enwiki-20220101-pages-articles-multistream-index.txt'
wiki_filename = '/afs/crc.nd.edu/group/dmsquare/vol1/data/WIKIPEDIA/wikipedia_dump/enwiki-20220101-pages-articles-multistream.xml.bz2'
start_byte, data_length = search_index(search_term, index_filename)
decompress_chunk(wiki_filename, start_byte, data_length)

'chunk.xml'

In [6]:
import bz2
from bs4 import BeautifulSoup

DUMP_FILE = wiki_filename
search_term = "Hawaii Technology Institute"
start_byte, data_length = search_index(search_term, index_filename)

decomp = bz2.BZ2Decompressor()
with open(DUMP_FILE, 'rb') as f:
    f.seek(start_byte)
    readback = f.read(data_length)
    page_xml = decomp.decompress(readback).decode()

OSError: Invalid data stream

In [14]:
page_xml[:50]
## "  <page>\n    <title>Stokell's smelt</title>\n    <n"
page_xml[-50:]
## '4bs3rqxoeidnvw67</sha1>\n    </revision>\n  </page>\n'

'z5h7ibbjsez2du81</sha1>\n    </revision>\n  </page>\n'

In [15]:
soup = BeautifulSoup(page_xml, "lxml")
pages = soup.find_all("page")
len(pages)

100

In [ ]:
page_titles = [p.find("title").text for p in pages]
page_index = page_titles.index("Donald Trump")


microbial_fuel_cell_text = pages[page_index].find("text").text
microbial_fuel_cell_text

In [26]:
from wikiextractor.extract import Extractor
extractor = Extractor("", "", "", "", "")

In [27]:
text = extractor.clean_text(microbial_fuel_cell_text)